In [1]:
import pandas as pd
import numpy as np
import joblib
import json
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import smps
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
from smps.fit import LogNormal
import plotly as py
import pyarrow.feather as feather
from math import sqrt
from sklearn.metrics import mean_squared_error
sns.set("notebook", "ticks", palette='colorblind') 
%matplotlib inline

In [2]:
folder = '/Users/zahrashivji/Dropbox/Shivji/Final Data/Outdoor Files/'
input_raw = folder + 'EST_Roof_Raw.csv'
input_final = folder + 'EST_Roof_Final.csv'

In [3]:
#Load the raw file 
df1 = dd.read_csv(input_raw, parse_dates = ['timestamp','timestamp_local']).compute()

# load the final data 
df_final = dd.read_csv(input_final, parse_dates=['timestamp','timestamp_local']).compute()
df2 = df_final[['raw_table_id','pm1','pm25','pm10']]

# merge the files
df = df1.merge(df2, left_on='id',right_on='raw_table_id')

In [4]:
#Drop the duplicated measurements
df = df.drop_duplicates(subset=['timestamp'])

In [5]:
# del unneccesary columns
del df['lat']
del df['lon']
del df['sample_pres'], df['device_state'], df['id']

In [6]:
# Set the index to be the timestamp_local
df.set_index("timestamp_local", inplace=True) 

# Only keep un-flagged data
df = df.query("flag == 0")

#Sort the timestamp_local so that it starts from lowest to highest date
df = df.sort_index()

#Specify start and end date of interest
t0= '2020-11-05'
tf= "2021-01-01"

# Keep only data between start and stop 
df = df[t0:tf]

In [7]:
# only keep rows where the values are positive
df = df[df['bin0'] >= 0.]
df = df[df['bin1'] >= 0.]
df = df[df['bin2'] >= 0.]
df = df[df['bin3'] >= 0.]
df = df[df['bin4'] >= 0.]
df = df[df['bin5'] >= 0.]
df = df[df['bin6'] >= 0.]
df = df[df['bin7'] >= 0.]
df = df[df['bin8'] >= 0.]
df = df[df['bin9'] >= 0.]
df = df[df['bin10'] >= 0.]
df = df[df['bin11'] >= 0.]
df = df[df['bin12'] >= 0.]
df = df[df['bin13'] >= 0.]
df = df[df['bin14'] >= 0.]
df = df[df['bin15'] >= 0.]
df = df[df['bin16'] >= 0.]
df = df[df['bin17'] >= 0.]
df = df[df['bin18'] >= 0.]
df = df[df['bin19'] >= 0.]
df = df[df['bin20'] >= 0.]
df = df[df['bin21'] >= 0.]
df = df[df['bin22'] >= 0.]
df = df[df['bin23'] >= 0.]
df = df[df['pm1'] >= 0.]
df = df[df['pm25'] >= 0.]
df = df[df['pm10'] >= 0.]

# identify the columns to keep
cols_to_keep = ['sample_temp', 'sample_rh','pm1', 'pm25', 'pm10','bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5',
                'bin6','bin7','bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13',
                'bin14','bin15','bin16', 'bin17', 'bin18', 'bin19', 'bin20', 'bin21',
                'bin22','bin23' ,'neph_bin0', 'neph_bin1', 'neph_bin2', 'neph_bin3', 
                'neph_bin4', 'neph_bin5', 'pm1_env', 'pm25_env', 'pm10_env']

# keep only certain columns
df = df[cols_to_keep]
df.index = df.index.tz_localize(None)

In [8]:
# Resample to a 5min time-base
df_6min = df.resample("6min").mean()
df_6min.reset_index()

,timestamp_local,sample_temp,sample_rh,pm1,pm25,pm10,bin0,bin1,bin2,bin3,...,bin23,neph_bin0,neph_bin1,neph_bin2,neph_bin3,neph_bin4,neph_bin5,pm1_env,pm25_env,pm10_env
0,2020-11-05 00:00:00,13.211667,52.680000,20.044017,20.536850,22.584833,12.914967,1.151983,0.323933,0.071100,...,0.0,2988.083333,889.138667,137.708333,6.388833,2.013833,0.652833,16.999833,23.639000,24.999833
1,2020-11-05 00:06:00,13.158333,52.618333,18.541850,19.061183,21.067967,12.583917,1.092117,0.309133,0.073817,...,0.0,2762.541667,826.486167,121.916667,5.583333,1.708167,0.264000,15.222167,21.055500,22.125000
2,2020-11-05 00:12:00,13.181667,52.328333,17.709983,18.128367,19.410350,12.070900,1.110283,0.294350,0.068950,...,0.0,2633.541667,791.124833,126.013833,5.889000,2.000000,0.277833,15.166500,21.041833,22.180500
3,2020-11-05 00:18:00,13.176667,51.965000,16.624933,17.148300,18.497650,11.691667,1.097817,0.309200,0.066183,...,0.0,2466.291667,738.250000,116.736000,6.264000,1.986167,0.638833,14.027667,19.986000,21.291667
4,2020-11-05 00:24:00,13.220000,51.506667,13.881683,14.361900,15.765650,11.416633,1.047700,0.283717,0.065817,...,0.0,2052.791667,611.458333,102.291500,6.153000,2.236167,0.777667,11.125167,16.333333,17.847167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,2021-01-01 23:30:00,17.923333,89.826667,4.543550,5.370967,6.144533,15.742250,1.868767,0.422067,0.104517,...,0.0,1455.125000,439.319500,86.930667,2.208333,0.930667,0.139000,8.139000,11.680667,12.375000
13916,2021-01-01 23:36:00,17.873333,90.008333,4.376300,5.204033,5.848083,15.923467,1.900750,0.395217,0.094917,...,0.0,1416.666667,433.874833,83.152833,3.527833,1.333333,0.458333,7.944333,11.514000,12.430333
13917,2021-01-01 23:42:00,17.820000,90.025000,4.540467,5.492667,6.314483,16.075750,1.932817,0.416300,0.101800,...,0.0,1472.833333,449.347167,91.583500,2.444500,0.722333,0.208500,8.430667,12.388833,12.777833
13918,2021-01-01 23:48:00,17.701667,90.330000,4.316117,5.288967,5.911850,16.161667,1.912083,0.410233,0.107250,...,0.0,1429.375000,435.513833,88.277833,2.680500,0.805667,0.139000,8.027833,11.972167,12.569500


In [9]:
df2 = pd.read_csv(
    folder + '20210225_SMPS_NumberDistributions_JCR.csv',
    skiprows = 23
)

def isfloat(str):
    try:
        float(str)
        return True
    except ValueError:
        return False

#Determine the total number of channels
n_channels = sum([isfloat(x) for x in df2.columns])

#Next determine the index of the first channel
channel0_idx = [i for i, x in enumerate([isfloat(x) for x in df2.columns]) if x][0]

#Convert to a datetime object
df2['timestamp']=df2.apply(lambda x: "{} {}".format(x['Date'],x['Start Time']),axis=1)
df2['timestamp']=df2['timestamp'].map(pd.to_datetime)

#Grab the bin diameters
midpoints = df2.columns[channel0_idx:channel0_idx+n_channels]
binlabels = ['bin{}'.format(i) for i in range(n_channels)]

#Rename the columns to bin<X>
df2.rename(columns=dict(zip(midpoints,binlabels)),inplace=True)

#Set the index for the data dataframe
df2.set_index('timestamp',inplace=True)

# df2.index = df2.index.tz_localize('US/Eastern')
df2.index = df2.index - pd.Timedelta(hours=1)

df2 = df2[t0:tf].copy()

# Resample to a 5 min timebase
df2 = df2.resample('6min').mean()

# Build out a nx3 array of the bin boundaries
bins = smps.utils.make_bins(
    midpoints=midpoints,
    lb = df2['Lower Size (nm)'][0],
    ub = df2['Upper Size (nm)'][0],
    channels_per_decade=64,
)

#B uild a generic Particle Sizer Object 
obj = smps.GenericParticleSizer(
    data=df2.copy(),
    bins=bins,
    fmt='dn',
    dp_units='nm',
    bin_labels=binlabels
)

In [10]:
#bias & RMSE calcs
tmp = obj.data[["Median (nm)", "Mean (nm)", "Mode (nm)", "Geo. Mean (nm)"]].copy()
tmp['pm1_neph'] = df_6min["pm1_env"]
tmp['pm1_modpm'] = df_6min["pm1"]
tmp["PM1_SMPS"] = obj.integrate(dmin=0, dmax=1., weight='mass', rho=1.65)
tmp['PM1_SMPS'].mask(tmp['PM1_SMPS'] == 0, np.nan, inplace = True) #replacing values where PM1_SMPS=0 with np.nan
tmp.dropna(inplace = True)
tmp['PMS PM1 bias'] = (df_6min["pm1_env"]-tmp["PM1_SMPS"])/tmp["PM1_SMPS"]
tmp['modpm PM1 bias'] = (df_6min["pm1"]-tmp["PM1_SMPS"])/tmp["PM1_SMPS"]
tmp['PMS PM1 RMSE'] = sqrt(mean_squared_error(tmp["PM1_SMPS"], tmp["pm1_neph"]))
tmp['modpm PM1 RMSE'] = sqrt(mean_squared_error(tmp["PM1_SMPS"], tmp["pm1_modpm"]))
tmp['rh'] = df_6min['sample_rh']

#### Bias plot groups

In [11]:
folder = '/Users/zahrashivji/Dropbox/Shivji/Final Data/Outdoor Files/'

df = df_6min

bin_list = ['bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10']

# group into 10
    # by RH
num_list = list(np.linspace(df['sample_rh'].min(), df['sample_rh'].max(), 11))
tmp['RH_key'] = df['sample_rh'].apply(lambda x: bin_list[0] if ((x >= num_list[0]) & (x <= num_list[1])) \
                                      else (bin_list[1] if ((x >= num_list[1]) & (x <= num_list[2])) \
                                      else (bin_list[2] if ((x >= num_list[2]) & (x <= num_list[3])) \
                                      else (bin_list[3] if ((x >= num_list[3]) & (x <= num_list[4])) \
                                      else (bin_list[4] if ((x >= num_list[4]) & (x <= num_list[5])) \
                                      else (bin_list[5] if ((x >= num_list[5]) & (x <= num_list[6])) \
                                      else (bin_list[6] if ((x >= num_list[6]) & (x <= num_list[7])) \
                                      else (bin_list[7] if ((x >= num_list[7]) & (x <= num_list[8])) \
                                      else (bin_list[8] if ((x >= num_list[8]) & (x <= num_list[9])) \
                                      else bin_list[9])))))))))
rh_bias = tmp.groupby('RH_key').mean()
rh_bias.to_csv(folder + 'rh_forIgor.csv')

    # by SMPS mass loading
num_list = list(np.linspace(tmp["PM1_SMPS"].min(), tmp["PM1_SMPS"].max(), 11))
tmp['ml_key'] = tmp["PM1_SMPS"].apply(lambda x: bin_list[0] if ((x >= num_list[0]) & (x <= num_list[1])) \
                                      else (bin_list[1] if ((x >= num_list[1]) & (x <= num_list[2])) \
                                      else (bin_list[2] if ((x >= num_list[2]) & (x <= num_list[3])) \
                                      else (bin_list[3] if ((x >= num_list[3]) & (x <= num_list[4])) \
                                      else (bin_list[4] if ((x >= num_list[4]) & (x <= num_list[5])) \
                                      else (bin_list[5] if ((x >= num_list[5]) & (x <= num_list[6])) \
                                      else (bin_list[6] if ((x >= num_list[6]) & (x <= num_list[7])) \
                                      else (bin_list[7] if ((x >= num_list[7]) & (x <= num_list[8])) \
                                      else (bin_list[8] if ((x >= num_list[8]) & (x <= num_list[9])) \
                                      else bin_list[9])))))))))
ml_bias = tmp.groupby('ml_key').mean()
ml_bias.to_csv(folder + 'ml_forIgor.csv')

    # by SMPS geo mean diameter
num_list = list(np.linspace(tmp["Geo. Mean (nm)"].min(), tmp["Geo. Mean (nm)"].max(), 11))
tmp['gm_key'] = tmp["Geo. Mean (nm)"].apply(lambda x: bin_list[0] if ((x >= num_list[0]) & (x <= num_list[1])) \
                                      else (bin_list[1] if ((x >= num_list[1]) & (x <= num_list[2])) \
                                      else (bin_list[2] if ((x >= num_list[2]) & (x <= num_list[3])) \
                                      else (bin_list[3] if ((x >= num_list[3]) & (x <= num_list[4])) \
                                      else (bin_list[4] if ((x >= num_list[4]) & (x <= num_list[5])) \
                                      else (bin_list[5] if ((x >= num_list[5]) & (x <= num_list[6])) \
                                      else (bin_list[6] if ((x >= num_list[6]) & (x <= num_list[7])) \
                                      else (bin_list[7] if ((x >= num_list[7]) & (x <= num_list[8])) \
                                      else (bin_list[8] if ((x >= num_list[8]) & (x <= num_list[9])) \
                                      else bin_list[9])))))))))
gm_bias = tmp.groupby('gm_key').mean()
gm_bias.to_csv(folder + 'gm_forIgor.csv')

In [22]:
# #plotting
# fig1 = px.scatter(
#     rh_1,
#     x= rh_1.index, y='PMS PM1 bias',
#     template = 'simple_white'
# )

# fig1.add_trace(
#     go.Scatter(x= rh_1.index, y=rh_1['modpm PM1 bias'], mode= 'markers'))

# fig2 = px.scatter(
#     rh_25,
#     x= rh_25.index, y='PMS PM1 bias',
#     template = 'simple_white'
# )

# fig2.add_trace(
#     go.Scatter(x= rh_25.index, y=rh_25['modpm PM1 bias'], mode= 'markers'))

# fig3 = px.scatter(
#     rh_5,
#     x= rh_5.index, y='PMS PM1 bias',
#     template = 'simple_white'
# )

# fig3.add_trace(
#     go.Scatter(x= rh_5.index, y=rh_5['modpm PM1 bias'], mode= 'markers'))

# fig4 = px.scatter(
#     rh_10,
#     x= rh_10.index, y='PMS PM1 bias',
#     template = 'simple_white'
# )

# fig4.add_trace(
#     go.Scatter(x= rh_10.index, y=rh_10['modpm PM1 bias'], mode= 'markers'))


# # fig2 = px.scatter(
# #     rh_1,
# #     x=rh_1.index, y='PMS PM1 RMSE',
# #     color_continuous_scale=px.colors.diverging.BrBG,
# #     template = 'simple_white'
# # )

# # fig2.add_trace(
# #     go.Scatter(x= rh_1.index, y=rh_1['modpm PM1 RMSE'], mode= 'markers'))

# fig5 = plt.figure(figsize=(12,6))
# # Make the figures big enough for the optically challenged.
# #pylab.rcParams['figure.figsize'] = (10.0, 8.0)
# # We start by making a copy of the original data frame, minus the sex attribute.
# #replot
# plot.boxplot(plot_array3)

NameError: name 'rh_1' is not defined